In [ ]:
import sys
import os

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
from forcateri.model.dartsmodels.dartstcnmodel import DartsTCNModel
from forcateri.baltbestapi.baltbestaggregatedapidata import BaltBestAggregatedAPIData
import pandas as pd
from forcateri.data.dataprovider import DataProvider, SeriesRole
from darts.models import TCNModel
from darts.utils.likelihood_models import QuantileRegression
from forcateri.data.timeseries import TimeSeries

In [ ]:
%load_ext autoreload
%autoreload 2 

In [ ]:
# baltbest = BaltBestAggregatedAPIData(
#     name='test', 
#     url="baltbest_url", 
#     local_copy="/home/user/DFKI/forcateri/_data/showcase_data.csv",
#     target = 'q_hca',
#     group_col = 'room_id',
#     time_col = 'datetime',
#     known = 'temperature_outdoor_avg',
#     observed = ['temperature_1_max', 'temperature_2_max','temperature_room_avg'],)
baltbest = BaltBestAggregatedAPIData(
    name='test', 
    url="baltbest_url", 
    local_copy="/home/user/DFKI/forcateri/_data/hourly_data.csv",
    target = 'delta',
    group_col = 'room_id',
    time_col = 'rounded_ts',
    known = 'outside_temp',
    observed = ['max_temperature_1', 'max_temperature_2','room_temperature'],)

In [ ]:
import logging

# Get the logger you used in your TimeSeries class
logger = logging.getLogger("forcateri.data.timeseries")
logger.setLevel(logging.INFO)  # or logging.DEBUG for more verbosity

# Create handler if it doesn't exist
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# Optional: prevent propagation to avoid double logs
logger.propagate = False


In [ ]:

#baltbest.get_data()


In [ ]:
# roles = {
#     'q_hca': SeriesRole.TARGET, 
#     'temperature_outdoor_avg':SeriesRole.KNOWN, 
#     'temperature_1_max':SeriesRole.OBSERVED, 
#     'temperature_2_max':SeriesRole.OBSERVED,
#     'temperature_room_avg':SeriesRole.OBSERVED,}
#['temperature_1_max', 'temperature_2_max','temperature_room_avg']
roles = {
    'delta': SeriesRole.TARGET, 
    'outside_temp':SeriesRole.KNOWN, 
    'max_temperature_1':SeriesRole.OBSERVED, 
    'max_temperature_1':SeriesRole.OBSERVED,
    'room_temperature':SeriesRole.OBSERVED,}

In [ ]:
start = pd.Timestamp(2021, 1, 3, 4,tz=0)
end = pd.Timestamp(2021, 1, 8, 4,tz=0)
dataprovider = DataProvider(data_sources=[baltbest], roles=roles, splits=(100,200))

In [ ]:
train = dataprovider.get_train_set()
val = dataprovider.get_val_set()
test = dataprovider.get_test_set()

In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
quantiles = [0.1, 0.5, 0.9]
dartstcn = DartsTCNModel( quantiles=quantiles)

In [ ]:
dartstcn.fit(train_data = train, val_data=val)

In [ ]:
num_forecast_steps = 2
#num_monte_carlo_samples = 50 # A sufficient number of samples to approximate the quantiles

prediction = dartstcn.predict(
    data=val, # Pass the series used for prediction context
    predict_likelihood_parameters = True,
    n=num_forecast_steps,
)

In [ ]:
from IPython.display import display, Markdown
def mprint(s): display(Markdown(s))

In [ ]:
prediction_ts = dartstcn.to_time_series(prediction)

In [ ]:
prediction_ts[0].data

In [ ]:
prediction[0].to_dataframe()